In [ ]:
! pip install requests
! pip install beautifulsoup4
! pip install openai
! pip install streamlit
! pip install scikit-learn
! pip install google-api-python-client
! pip install selenium
! pip install webdriver_manager

In [16]:
# prompt: code that will pull a html page from the web and phase is


def get_web_page(url):
    import requests
    from bs4 import BeautifulSoup

    # Define the URL of the web page you want to scrape

    # Send a GET request to the URL and get the response
    response = requests.get(url)
    paragraphs = []
    # Check for successful response
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the paragraph elements on the page
        paragraphs = soup.find_all('p')

        # # Print the text of each paragraph
        # for paragraph in paragraphs:
        #     print(paragraph.text)
    else:
        # Handle error
        print(f"Error fetching the web page: {response.status_code}")
    
    return paragraphs

paragraphs = get_web_page("https://a.co/d/7Z6Ne4i")

In [17]:
# Create a prompt that contains the paragraphs
prompt = """
extract list of features of this product for each features, categorized the persona that will use it.
response in json

{
  "product_name":<the name of the product>,
  "identifier":<if applicable, the manafactor product id>,
  "characteristics": [{
  "feature": <one of the features of the product>
  "personns": [
    <one of the persona that will need this feauture>,
    // add more here
  ]

}]
}
          """
for paragraph in paragraphs:
    prompt += paragraph.text + "\n"

# Call OpenAI with the prompt
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=False,
)
print(stream.choices[0].message.content)

{
  "product_name": "MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey",
  "identifier": "MK-Box",
  "characteristics": [
    {
      "feature": "Dimension: 30.8*10.1*3.9cm, 12.1*4*1.5in",
      "personas": [
        "Gamer",
        "Office Worker"
      ]
    },
    {
      "feature": "Net Weight: 570g,1.26lb",
      "personas": [
        "Traveler",
        "Student"
      ]
    },
    {
      "feature": "Keys: 68 keys",
      "personas": [
        "Gamer",
        "Typist"
      ]
    },
    {
      "feature": "Switch: red switch",
      "personas": [
        "Gamer",
        "Programmer"
      ]
    },
    {
      "feature": "Connection: detachable type-c wired",
      "personas": [
        "Tech Enthusiast",
        "Working Professional"
      ]
    },
    {
      "feature": "Keycap: ergonomics ABS double-shot injection",
      "personas": [
        "Gamer",
      

In [18]:
import json

product_persons = json.loads(stream.choices[0].message.content.replace("json","").replace("```",""))
def extract_sets_of_kwargs(data):
    sets_of_kwargs = []
    for characteristic in data['characteristics']:
        kwargs = {
            'product_name': data['product_name'],
            'identifier': data['identifier'],
            'feature': characteristic['feature']
        }
        sets_of_kwargs.append(kwargs)
    return sets_of_kwargs

sets_of_kwargs = extract_sets_of_kwargs(product_persons)
for kwargs_set in sets_of_kwargs:
    print(kwargs_set)

{'product_name': 'MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey', 'identifier': 'MK-Box', 'feature': 'Dimension: 30.8*10.1*3.9cm, 12.1*4*1.5in'}
{'product_name': 'MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey', 'identifier': 'MK-Box', 'feature': 'Net Weight: 570g,1.26lb'}
{'product_name': 'MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey', 'identifier': 'MK-Box', 'feature': 'Keys: 68 keys'}
{'product_name': 'MageGee Portable 60% Mechanical Gaming Keyboard, MK-Box LED Backlit Compact 68 Keys Mini Wired Office Keyboard with Red Switch for Windows Laptop PC Mac - Black/Grey', 'identifier': 'MK-Box', 'feature': 'Switch: red switch'}
{'prod

In [22]:
prompt = """
formulate 5 google search queries that will help a buyer learn about the product '{product_name}' {identifier}

response in a json starcture:
[
 <first search engine query>,
 // add here more
]
"""

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt.format(**sets_of_kwargs[0])}],
    stream=False,
)
print(stream.choices[0].message.content)

[
 "MageGee Portable 60% Mechanical Gaming Keyboard MK-Box review",
 "MageGee Portable MK-Box keyboard features",
 "Where to buy MageGee Portable MK-Box keyboard",
 "MageGee Portable MK-Box keyboard compatibility",
 "MageGee Portable MK-Box keyboard comparison with other similar products"
]


In [23]:
queries = json.loads(stream.choices[0].message.content.replace("json","").replace("```",""))
queries

['MageGee Portable 60% Mechanical Gaming Keyboard MK-Box review',
 'MageGee Portable MK-Box keyboard features',
 'Where to buy MageGee Portable MK-Box keyboard',
 'MageGee Portable MK-Box keyboard compatibility',
 'MageGee Portable MK-Box keyboard comparison with other similar products']

In [24]:

def _google_search_results(search_term: str, **kwargs) :
    from googleapiclient.discovery import build
    import os

    google_cse_id= os.environ["google_cse_id"]
    google_api_key = os.environ["google_api_key"] 

    search_engine = build("customsearch", "v1", developerKey=google_api_key)
    cse = search_engine.cse()
    # if self.siterestrict:
    #     cse = cse.siterestrict()
    res = cse.list(q=search_term, cx=google_cse_id, **kwargs).execute()
    return res.get("items", [])

search_results = dict()
for query in queries:
    search_results[query] = _google_search_results(query)

In [25]:

def selenium_get(
    url,
    headless=True,
):
    """start browser"""
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from bs4 import BeautifulSoup

    executable_path = ChromeDriverManager().install()
    chrome_options = webdriver.ChromeOptions()

    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--lang=en")
    chrome_options.add_argument("ignore-certificate-errors")

    if headless:
        chrome_options.headless = True
        chrome_options.add_argument("--headless")

    service = Service(executable_path=executable_path)
    webdriver_client = None
    try:
        webdriver_client = webdriver.Chrome(service=service, options=chrome_options)
        webdriver_client.get(url)

            
        soup = BeautifulSoup(webdriver_client.page_source, 'html.parser')
        return soup.find_all('p')
    finally:
        if webdriver_client:
            webdriver_client.close()
        
    
#selenium_get("https://www.n12.co.il/")


In [26]:
def web_page_if_not_selenium(url):

    response = get_web_page(url)

    if len(response) == 0:
        return selenium_get(url)
    return response

In [27]:
pages_to_learn = list()
processed_urls = list()
for query in queries:
    for response in search_results[query]:

        url_to_open = response["formattedUrl"]
        if url_to_open not in processed_urls:
            pages_to_learn.append(web_page_if_not_selenium(url_to_open))
            processed_urls.append(url_to_open)

Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 444
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 503
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 444
Error fetching the web page: 404
Error fetching the web page: 503
Error fetching the web page: 503
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 403
Error fetching the web page: 403
Error fetching the web page: 404
Error fetching the web page: 404
Error fetching the web page: 503


In [28]:
pages_to_learn

[[],
 [<p class="px-md pt-md m-0 whitespace-normal text-14 leading-5 font-normal">
                Non-mouse and non-audio input devices for the discerning fingertip.
              </p>,
  <p>
      Hi im fairly new to the keyboard community. Looking for my first mechanical keyboard and was wondering if magegee is a good brand? If so, is the mk1/mk star worth buying?
    </p>],
 [],
 [],
 [],
 [<p id="notify_error" style="color: #F94141;"></p>,
  <p>from one-hit wonders</p>,
  <p>from reviewers with more than one review</p>,
  <p>
  <img class="pull-left img-responsive" data-src="/public/imgs/reviewmeta/rs-prod-icon.png" src="/public/imgs/reviewmeta/placeholder-10.jpg" style="margin-right: 10px;"/>
  <b><u>4.6</u></b> Product's Average Ease Score
  						<span class="clearfix"></span>
  </p>,
  <p>
  <img class="pull-left img-responsive" data-src="/public/imgs/reviewmeta/rs-cat-icon.png" src="/public/imgs/reviewmeta/placeholder-10.jpg" style="margin-right:10px;"/>
  <b><u>4.3</u></b> C

In [12]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

df = pd.DataFrame(data={"persona":persons_list})
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.persona.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('basic_persona_v1.csv', index=False)

In [120]:
def search_by_persona(input_text):
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np
    import ast
    from numpy import dot
    from numpy.linalg import norm


    data = pd.read_csv('basic_persona_v1.csv')
    data["ada_embedding"] = data["ada_embedding"].apply(lambda x: ast.literal_eval(x))

    # Calculate cosine similarity
    input_embedding = get_embedding(input_text)


    # Calculate cosine similarity
    similarities = data['ada_embedding'].apply(lambda x:dot(x, input_embedding)/(norm(x)*norm(input_embedding)))

    return data.iloc[similarities.sort_values().index,:]['persona']


In [122]:
# prompt: get user input
name = input("Tell me about you self?")
print(f"User input is: {name}")
ordered_res = search_by_persona(name)
print(ordered_res)

User input is: I love taveling
3           Gamers looking for a responsive switch
5                Users looking for durable keycaps
1         Users looking for a lightweight keyboard
2    Gamers who prefer a minimalist keyboard setup
6     Users who want customizable lighting options
0              Users who prefer a compact keyboard
4      Users who value convenience and portability
Name: persona, dtype: object
